In [2]:
import os
import numpy as np
np.set_printoptions(suppress=True)

from scipy.spatial.transform import Rotation

import poselib

In [12]:
x2d = np.array([192.12533569, 19.14378548])
camera = {'f': 277.4716064453125, 'cx': 160.0, 'cy': 90.0, 'w': 320, 'h': 180}
x3d = np.array([ 11.86180782, -14.56327057,  -0.92378181])

In [13]:
R = Rotation.random().as_matrix()
c = np.random.rand(3)

In [14]:
t = -R @ c

In [15]:
proj = R @ x3d + t

In [16]:
proj

array([  5.02603999, -16.25407961,  -8.13491714])

In [17]:
R.T @ (proj - t)

array([ 11.86180782, -14.56327057,  -0.92378181])

In [18]:
proj = R @ (x3d - t)

In [19]:
proj

array([  3.86514379, -16.66311341,  -7.9395824 ])

In [20]:
R.T @ proj + t

array([ 11.86180782, -14.56327057,  -0.92378181])

## There is two ways to apply extrinsics

In [3]:
x = np.array([1., 1., 1.])

In [4]:
R = Rotation.random().as_matrix()
c = np.random.rand(3)

In [5]:
R, c

(array([[ 0.03356652, -0.33598691, -0.94126834],
        [ 0.00402645, -0.94174595,  0.33630098],
        [-0.99942837, -0.01507843, -0.0302583 ]]),
 array([0.14032744, 0.60229413, 0.03921471]))

In [6]:
x2d_t = R @ (x - c) # translate first then rotate

In [7]:
x2d_t

array([-1.00912451, -0.04796343, -0.89424966])

In [8]:
R.T @ x2d_t + c

array([1., 1., 1.])

In [9]:
t = - R @ c

In [10]:
x2d_r = R @ x + t # rotate first then translate

In [11]:
x2d_r

array([-1.00912451, -0.04796343, -0.89424966])

In [11]:
R.T @ (x2d_r - t)

array([1., 1., 1.])

## Projections!

In [22]:
# those taken from ground truth
c = np.array([-1.495227, 0.04562, -10.339456])
q = np.array([0.901882, -0.251623, -0.34861, 0.042019])

# note the inconsistency

qmtx = Rotation.from_quat([*q[1:], q[0]]) # x, y, z, w
qmtx.as_matrix()

array([[ 0.75341089,  0.09964425, -0.64995619],
       [ 0.251229  ,  0.86984051,  0.4245721 ],
       [ 0.60766439, -0.48316509,  0.6303138 ]])

In [23]:
gt_pose = poselib.CameraPose()
gt_pose.q = q

In [24]:
R = gt_pose.R

In [25]:
# example pair taken from the experiment
x = np.array([192.12533569, 19.14378548])
X = np.array([11.86180782, -14.56327057, -0.92378181])

In [26]:
t = -R @ c

In [27]:
R, t

(array([[ 0.75341094,  0.09964423, -0.64995606],
        [ 0.25122895,  0.86984054,  0.42457202],
        [ 0.60766427, -0.483165  ,  0.63031387]]),
 array([-5.59821749,  4.72580592,  7.44774052]))

In [28]:
# SIMPLE_RADIAL
camera = {'f': 277.4716064453125, 'cx': 160.0, 'cy': 90.0, 'w': 320, 'h': 180}

In [29]:
def pix2cam(x, camera):
    x = np.concatenate([x, np.ones(1)])
    x[0] -= camera["cx"]
    x[1] -= camera["cy"]
    x[:2] /= camera["f"]
    x /= np.linalg.norm(x)
    
    return x

def cam2pix(x, camera):
    x /= x[2]
    x[:2] *= camera["f"]
    x[0] += camera["cx"]
    x[1] += camera["cy"]
    
    return x

def w2c(X, R, t):
    return R @ X + t

cam2pix(w2c(X.copy(), R, t), camera)

array([192.70088458,  19.62502469,   1.        ])

In [32]:
cam2pix(pix2cam(x.copy(), camera), camera)[:2]

array([192.12533569,  19.14378548])

In [33]:
X

array([ 11.86180782, -14.56327057,  -0.92378181])

In [34]:
x, pix2cam(x.copy(), camera)

(array([192.12533569,  19.14378548]),
 array([ 0.11147971, -0.24588163,  0.96286785]))

In [35]:
w2c(X.copy(), R, t), cam2pix(w2c(X.copy(), R, t), camera)

(array([ 2.48787005, -5.35409962, 21.10992744]),
 array([192.70088458,  19.62502469,   1.        ]))

In [36]:
'''
// params = f, cx, cy, k1

void SimpleRadialCameraModel::project(const std::vector<double> &params, const Eigen::Vector2d &x,
                                      Eigen::Vector2d *xp) {
    const double r2 = x.squaredNorm();
    const double alpha = (1.0 + params[3] * r2);
    (*xp)(0) = params[0] * alpha * x(0) + params[1];
    (*xp)(1) = params[0] * alpha * x(1) + params[2];
}

void SimpleRadialCameraModel::unproject(const std::vector<double> &params, const Eigen::Vector2d &xp,
                                        Eigen::Vector2d *x) {
    (*x)(0) = (xp(0) - params[1]) / params[0];
    (*x)(1) = (xp(1) - params[2]) / params[0];
    double r0 = x->norm();
    double r = undistort_poly1(params[3], r0);
    (*x) *= r / r0;
}
'''

'\n// params = f, cx, cy, k1\n\nvoid SimpleRadialCameraModel::project(const std::vector<double> &params, const Eigen::Vector2d &x,\n                                      Eigen::Vector2d *xp) {\n    const double r2 = x.squaredNorm();\n    const double alpha = (1.0 + params[3] * r2);\n    (*xp)(0) = params[0] * alpha * x(0) + params[1];\n    (*xp)(1) = params[0] * alpha * x(1) + params[2];\n}\n\nvoid SimpleRadialCameraModel::unproject(const std::vector<double> &params, const Eigen::Vector2d &xp,\n                                        Eigen::Vector2d *x) {\n    (*x)(0) = (xp(0) - params[1]) / params[0];\n    (*x)(1) = (xp(1) - params[2]) / params[0];\n    double r0 = x->norm();\n    double r = undistort_poly1(params[3], r0);\n    (*x) *= r / r0;\n}\n'

```c++
const Eigen::Matrix3d R = pose.R();
const double P0_0 = R(0, 0), P0_1 = R(0, 1), P0_2 = R(0, 2), P0_3 = pose.t(0);
const double P1_0 = R(1, 0), P1_1 = R(1, 1), P1_2 = R(1, 2), P1_3 = pose.t(1);
const double P2_0 = R(2, 0), P2_1 = R(2, 1), P2_2 = R(2, 2), P2_3 = pose.t(2);

const double X0 = X[k](0), X1 = X[k](1), X2 = X[k](2);
const double x0 = x[k](0), x1 = x[k](1);

// P00 P01 P02 P03 | X0 
// P10 P11 P12 P13 | X1 
// P20 P21 P22 P23 | X2
// 0   0   0   0   | 1

const double z0 = P0_0 * X0 + P0_1 * X1 + P0_2 * X2 + P0_3;
const double z1 = P1_0 * X0 + P1_1 * X1 + P1_2 * X2 + P1_3;
const double z2 = P2_0 * X0 + P2_1 * X1 + P2_2 * X2 + P2_3;
const double inv_z2 = 1.0 / z2;

const double r_0 = z0 * inv_z2 - x0;
const double r_1 = z1 * inv_z2 - x1;
const double r_sq = r_0 * r_0 + r_1 * r_1;
```

## Prerotation composition

In [3]:
Ry = np.array([[-0.67087596, 0., -0.74156958], 
 [0. ,         1. ,     0.        ],
 [ 0.74156958,  0. ,        -0.67087596]])

Rz = np.array([[ 0.21006082, 0.97768832, 0.],
 [-0.97768832,  0.21006082, -0.      ],
 [-0.  ,        0.,          1.  ,      ]])

In [70]:
Ry

array([[-0.67087596,  0.        , -0.74156958],
       [ 0.        ,  1.        ,  0.        ],
       [ 0.74156958,  0.        , -0.67087596]])

In [71]:
Rz

array([[ 0.21006082,  0.97768832,  0.        ],
       [-0.97768832,  0.21006082, -0.        ],
       [-0.        ,  0.        ,  1.        ]])

In [4]:
R = Ry @ Rz

In [5]:
R

array([[-0.14092475, -0.65590759, -0.74156958],
       [-0.97768832,  0.21006082,  0.        ],
       [ 0.15577471,  0.72502392, -0.67087596]])

In [6]:
Rotation.from_matrix(Rz).as_euler("XYZ", degrees=True)

array([  0.       ,   0.       , -77.8740835])

In [7]:
Rotation.from_matrix(Ry).as_euler("XYZ", degrees=True)

array([ 180.        ,  -47.86529197, -180.        ])

In [8]:
Rotation.from_matrix(R).as_euler("XYZ", degrees=True)

array([180.        , -47.86529197, 102.1259165 ])

In [9]:
X = np.array([1, 1, 1])

In [12]:
Ry @ (Rz @ X)

array([-1.53840192, -0.7676275 ,  0.20992267])

In [13]:
R @ X

array([-1.53840192, -0.7676275 ,  0.20992267])

## Gimbal lock ?

In [15]:
Rotation.from_euler("XYZ", [-180, 0, 180], degrees=True).as_matrix()

array([[-1., -0.,  0.],
       [-0.,  1.,  0.],
       [-0.,  0., -1.]])

In [43]:
Rotation.from_euler("XYZ", [180, 0, -180], degrees=True).as_matrix() 

array([[-1.,  0.,  0.],
       [ 0.,  1., -0.],
       [-0., -0., -1.]])

In [20]:
Rotation.from_euler("XYZ", [0, 0, 0], degrees=True).as_matrix()

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [18]:
Rotation.from_euler("XYZ", [-0, 0, -0], degrees=True).as_matrix()

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [24]:
Rotation.from_euler("XYZ", [0, 0, 0], degrees=True).as_matrix()

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [38]:
Rotation.from_euler("XYZ", [0, 0, 180], degrees=True).as_matrix()

array([[-1., -0.,  0.],
       [ 0., -1.,  0.],
       [ 0.,  0.,  1.]])

In [29]:
Rotation.from_euler("XYZ", [0., 0., 179.77] , degrees=True).as_matrix()

array([[-0.99999194, -0.00401425,  0.        ],
       [ 0.00401425, -0.99999194,  0.        ],
       [ 0.        ,  0.        ,  1.        ]])

In [42]:
R1 = Rotation.from_euler("XYZ", [0, 0, 0], degrees=True).as_matrix()
R2 = Rotation.from_euler("XYZ", [180, 0, -180], degrees=True).as_matrix()

rot_error = np.arccos((np.trace(np.matmul(R1, R2)) - 1.0) / 2.0) * 180.0 / np.pi
rot_error

180.0

In [44]:
R1 = np.array([[-0.96874633, 0., 0.24805353], 
 [ 0.,          1.,          0.,        ], 
 [-0.24805353,  0.,         -0.96874633]])

In [45]:
R2 = np.array(
[[ 0.99619472,  0.,         -0.08715574],
 [ 0.         , 1. ,         0.        ],
 [ 0.08715574  ,0.  ,        0.99619472]]
)

In [46]:
R1, R2

(array([[-0.96874633,  0.        ,  0.24805353],
        [ 0.        ,  1.        ,  0.        ],
        [-0.24805353,  0.        , -0.96874633]]),
 array([[ 0.99619472,  0.        , -0.08715574],
        [ 0.        ,  1.        ,  0.        ],
        [ 0.08715574,  0.        ,  0.99619472]]))

In [71]:
fixer = np.eye(3)
fixer[0, 0] = -1
fixer[2, 2] = -1
Rotation.from_matrix(R1 @ fixer).as_euler("XYZ", degrees=True)

array([  0.        , -14.36235994,   0.        ])

In [65]:
Rotation.from_matrix(R2).as_euler("XYZ", degrees=True)

array([ 0.        , -4.99999979,  0.        ])

In [52]:
rot_error = np.arccos((np.trace(np.matmul(R1, R2)) - 1.0) / 2.0) * 180.0 / np.pi
rot_error

160.6376443249159

In [59]:
np.linalg.det(R1)

1.0000000056339298

In [60]:
np.linalg.det(R2)

1.000000043170826